# STEP 0 + 1
Merge dello step 0 e e 1 -> modifica del codice dello step 0 per far si che consideri anche le QPU e in output non dica solo come tagliare ma anche in quale QPU eseguire i frammenti.

## Definizione dei parametri
QPUs
Di ogni QPU abbiamo
- tempo di esecuzione
- tempo di coda
- capienza di qbit 

Circuito rappresentato come DAG
- archi
- vertici
- peso dei vertici -> qbit richiesto per vertice

In [68]:

# definizione del circuito
edges = [
    (0, 1), (1, 2),
    (2, 3), (3, 4),
    (4, 5), (5, 6),
    (6, 7), (7, 8),
    (8, 9),
]
vertices = set()
for edge in edges:
    vertices.update(edge)

vertex_weights = {v: 1 for v in vertices}  # 1 qubit per gate in questo esempio

# definizione delle QPUs

class QPU: 
    def __init__(self, nome, tempo_di_esecuzione, tempo_di_coda, capacita, index):
        self.nome = nome
        self.tempo_di_esecuzione = tempo_di_esecuzione
        self.tempo_di_coda = tempo_di_coda
        self.capacita = capacita
        self.index = index

nomi = ['qpu_0', 'qpu_1', 'qpu_2', 'qpu_3', 'qpu_4']
tempo_di_esecuzione = [10, 200, 150, 300, 10]
tempo_di_coda       = [ 1,  4,  3,  1,  3]
capacita            = [ 4,  4,  4,  4,  4]

qpus = []
for i in range(len(nomi)):
    qpus.append(QPU(nomi[i], tempo_di_esecuzione[i], tempo_di_coda[i], capacita[i], i))

# parametri per il CutQC dipendenti dalle QPU disponibili
num_subcircuits = len(qpus)  # Stessa dimensione dell'array QPU
num_qpus = len(qpus)
qpus_index = range(num_qpus) # indici dei qpu
subcircuits = range(num_subcircuits) # indici dei sottocircuiti


## Definizione delle variabili principali

In [69]:
import pulp

problem = pulp.LpProblem("CircuitCutter_WithQPU", pulp.LpMinimize)


# y[v,c]: se il gate v appartiene al sottocircuito c
y = pulp.LpVariable.dicts("y",
    [(v, c) for v in vertices for c in subcircuits],
    cat=pulp.LpBinary)

# x[e,c]: se l'arco e è tagliato dal sottocircuito c
x = pulp.LpVariable.dicts("x",
    [(e, c) for e in edges for c in subcircuits],
    cat=pulp.LpBinary)

# a[c]: numero di qubit/gate originali inclusi in sottocircuito c
# p[c]: qubit di inizializzazione
# o[c]: qubit misurati in uscita
# f[c]: qubit che contribuiscono alla misura finale
# d[c]: numero totale di qubit in input al sottocircuito d[c] = a[c] + p[c]
a = pulp.LpVariable.dicts("a", subcircuits, cat=pulp.LpInteger)
p = pulp.LpVariable.dicts("p", subcircuits, cat=pulp.LpInteger)
o = pulp.LpVariable.dicts("o", subcircuits, cat=pulp.LpInteger)
f = pulp.LpVariable.dicts("f", subcircuits, cat=pulp.LpInteger)
d = pulp.LpVariable.dicts("d", subcircuits, cat=pulp.LpInteger)

# variabili per linearizzare i prodotti
# z_p[e, c]:  x[e,c] * y[e[1], c] 
# z_o[e, c]:  x[e,c] * y[e[0], c]
z_p = pulp.LpVariable.dicts("z_p",
    [(e, c) for e in edges for c in subcircuits],
    cat=pulp.LpBinary)
z_o = pulp.LpVariable.dicts("z_o",
    [(e, c) for e in edges for c in subcircuits],
    cat=pulp.LpBinary)

# variabile nuova rispetto allostep 0 per includere le QPUs
# assign[c,q] = 1 se il sottocircuito c è assegnato alla QPU q
assign = pulp.LpVariable.dicts("assign",
    [(c, q) for c in subcircuits for q in qpus_index],
    cat=pulp.LpBinary)


## Definizione dei vincoli

In [70]:
# vincoli su a[c], p[c], o[c], f[c], d[c]
for c in subcircuits:
    problem += a[c] == pulp.lpSum(vertex_weights[v]*y[v,c] for v in vertices), f"A_{c}"
    problem += p[c] == pulp.lpSum(z_p[(e,c)] for e in edges), f"P_{c}"
    problem += o[c] == pulp.lpSum(z_o[(e,c)] for e in edges), f"O_{c}"
    problem += f[c] == a[c] + p[c] - o[c], f"F_{c}"
    problem += d[c] == a[c] + p[c], f"D_{c}"

#linearizzazione per z_p ed z_o
for e in edges:
    for c in subcircuits:
        # z_p = x[e,c] * y[e[1], c]
        problem += z_p[(e, c)] <= x[(e, c)],           f"zp_1_{e}_{c}"
        problem += z_p[(e, c)] <= y[(e[1], c)],        f"zp_2_{e}_{c}"
        problem += z_p[(e, c)] >= x[(e, c)] + y[(e[1], c)] - 1, f"zp_3_{e}_{c}"

        # z_o = x[e,c] * y[e[0], c]
        problem += z_o[(e, c)] <= x[(e, c)],           f"zo_1_{e}_{c}"
        problem += z_o[(e, c)] <= y[(e[0], c)],        f"zo_2_{e}_{c}"
        problem += z_o[(e, c)] >= x[(e, c)] + y[(e[0], c)] - 1, f"zo_3_{e}_{c}"

# ogni vertice deve stare in esattamente un sottocircuito
for v in vertices:
    problem += pulp.lpSum(y[v,c] for c in subcircuits) == 1, f"Vertice_{v}_unico"

# vincoli sugli archi (no tagli se e[0] e e[1] sono nello stesso sottocircuito)
for c in subcircuits:
    for e in edges:
        problem += x[e,c] <= y[e[0],c] + y[e[1],c],         f"x_1_{e}_{c}"
        problem += x[e,c] >= y[e[0],c] - y[e[1],c],         f"x_2_{e}_{c}"
        problem += x[e,c] >= y[e[1],c] - y[e[0],c],         f"x_3_{e}_{c}"
        problem += x[e,c] <= 2 - y[e[0],c] - y[e[1],c],     f"x_4_{e}_{c}"

# vincolo per l'ordine
for k in range(num_subcircuits):
    problem += pulp.lpSum(y[(k, j)] for j in range(k+1, num_subcircuits)) == 0, f"Ordine_subc_{k}"

#NUOVI VINCOLI
# ogni sottocircuito sta su una sola QPU
for c in subcircuits:
    problem += pulp.lpSum(assign[c,q] for q in qpus_index) == 1, f"Assegnamento_unico_{c}"

# vincolo di capacità: d[c] <= capacita della QPU a cui è assegnato c
for c in subcircuits:
    problem += d[c] <= pulp.lpSum(assign[c,q]*qpus[q].capacita for q in qpus_index), f"Capacita_{c}"

# una QPU non può contenere più di un sottocircuito
for q in qpus_index:
    problem += pulp.lpSum(assign[c,q] for c in subcircuits) <= 1, f"Max_1_subc_sulla_QPU_{q}"

## Fuznione obiettivo

A differenza di CutQC in questo caso la funzione obiettivo tiene conto sia dei tagli sia dei tempi di esecuzione. 
Ho deciso di minimizzare una combinazione lineare tra la minimizzazione dei tagli e la grandezza max⁡{tempo di coda+tempo di esecuzione}delle QPU.

In questo modo si cerca un compromesso tra il numero di tagli e i ritardi dovuti a coda ed esecuzione, così da ottimizzare  l’intero processo.
Minimizzare i tagli = sum(x[e,c]) / 2 = K  
Minimizzare il makespan dei qpu seelzionati = T

min {alpha * K + beta * T}

in base all'importanza di una funzione o l'altra si aumenta o diminuisce un parametro

Correzione:
Ho normalizzato le due variabili K  e T  proiettandole su un intervallo comune [0,1]

In [71]:
alpha = 1  # peso per il numero di tagli
beta = 1  # peso per il tempo di esecuzione

# makespan
T = pulp.LpVariable("Massimo_tempo_esecuzione_+_tempo_coda", lowBound=0, cat=pulp.LpContinuous)

# per ogni QPU q, imponiamo T >= "tempo se la QPU q viene usata"
for q in qpus_index:
    problem += T >= pulp.lpSum(assign[c, q] * (qpus[q].tempo_di_esecuzione + qpus[q].tempo_di_coda)
                               for c in subcircuits), f"Makespan_QPU_{q}"

# funzione obiettivo del CutQC
K = pulp.lpSum(x[e,c] for c in subcircuits for e in edges) / 2

# calcolo massimi per la normalizzazione
K_max = len(edges) / 2  # numero massim di tagli = numero di qbit
T_max = max(q.tempo_di_esecuzione + q.tempo_di_coda for q in qpus)  # tempo massimo

# normalizzazione
K_norm = K / K_max 
T_norm = T * (1 / T_max) # unsupported operand type(s) for /: 'LpVariable' and 'int' => ho risolto con molitplicazione con reciproco

# funzione obiettivo normalizzata
problem += alpha * K_norm + beta * T_norm, "Minimizza_Tagli_e_Makespan_Normalizzati"

problem.solve()

1

## Stampa risultati

In [72]:
print(f"Status: {pulp.LpStatus[problem.status]}")
print(f"Valore obiettivo: {pulp.value(problem.objective)}\n")

# sssegnazione dei vertici ai sottocircuiti
print("Assegnazione vertici -> sottocircuiti:")
for v in sorted(vertices):
    for c in subcircuits:
        if pulp.value(y[(v,c)]) == 1:
            print(f"  Vertice {v} nel sottocircuito {c}")
print()

# tagli sugli archi
print("Tagli sugli archi:")
for e in edges:
    for c in subcircuits:
        if pulp.value(x[(e,c)]) == 1:
            print(f"  Arco {e} tagliato dal sottocircuito {c}")
print()

# valori di a[c] p[c] o[c] f[c] d[c] per ogni sottocircuito
print("Qubit per sottocircuito:")
for c in subcircuits:
    a_val = pulp.value(a[c])
    p_val = pulp.value(p[c])
    o_val = pulp.value(o[c])
    f_val = pulp.value(f[c])
    d_val = pulp.value(d[c])
    print(f"  Sottocircuito {c}:")
    if a_val == 0:
        print(f"    Vuoto")
        continue
    print(f"    a[{c}] = {a_val}  (gate/qubit originali)")
    print(f"    p[{c}] = {p_val}  (qubit di inizializzazione)")
    print(f"    o[{c}] = {o_val}  (qubit misurati in uscita)")
    print(f"    f[{c}] = {f_val}  (qubit che contribuiscono alla misura finale)")
    print(f"    d[{c}] = {d_val}  (totale qubit in input)")
    print()

# assegnamento dei sottocircuiti alle QPU
print("Assegnamento sottocircuiti -> QPU:")
for c in subcircuits:
    for q in qpus_index:
        if pulp.value(assign[(c, q)]) == 1:
            nome_qpu = qpus[q].nome
            cap_qpu = qpus[q].capacita
            tempo_totale = qpus[q].tempo_di_esecuzione +  qpus[q].tempo_di_coda

            if pulp.value(a[c]) != 0:  
                print(f"  Sottocircuito {c} assegnato a {nome_qpu} (capacità {cap_qpu}, tempo totale: {tempo_totale})")
print()

# stampa dei sottocircuiti
print("Sottocircuiti (archi assegnati):")
for c in subcircuits:
    archi_sottocircuito = [e for e in edges if pulp.value(y[e[0], c]) == 1 and pulp.value(y[e[1], c]) == 1]
    if archi_sottocircuito:  # stampa solo sottocircuiti non vuoti
        print(f"  Sottocircuito {c}: {archi_sottocircuito}")
print()

print("Tempo totale: ")
max_value = -1
for c in subcircuits:
    for q in qpus_index:
        if pulp.value(assign[(c, q)]) == 1:
            nome_qpu = qpus[q].nome
            cap_qpu = qpus[q].capacita
            tempo_totale = qpus[q].tempo_di_esecuzione +  qpus[q].tempo_di_coda
            if pulp.value(a[c]) != 0:  
                print(f"  Sottocircuito {c} assegnato a {nome_qpu} (capacità {cap_qpu}, tempo totale: {tempo_totale})")
                if max_value < tempo_totale:
                    max_value = tempo_totale
print()
print(f"Totale tempo: {max_value}")


Status: Optimal
Valore obiettivo: 1.4444444444444444

Assegnazione vertici -> sottocircuiti:
  Vertice 0 nel sottocircuito 0
  Vertice 1 nel sottocircuito 0
  Vertice 2 nel sottocircuito 0
  Vertice 3 nel sottocircuito 0
  Vertice 4 nel sottocircuito 3
  Vertice 5 nel sottocircuito 3
  Vertice 6 nel sottocircuito 3
  Vertice 7 nel sottocircuito 4
  Vertice 8 nel sottocircuito 4
  Vertice 9 nel sottocircuito 4

Tagli sugli archi:
  Arco (3, 4) tagliato dal sottocircuito 0
  Arco (3, 4) tagliato dal sottocircuito 3
  Arco (6, 7) tagliato dal sottocircuito 3
  Arco (6, 7) tagliato dal sottocircuito 4

Qubit per sottocircuito:
  Sottocircuito 0:
    a[0] = 4.0  (gate/qubit originali)
    p[0] = 0.0  (qubit di inizializzazione)
    o[0] = 1.0  (qubit misurati in uscita)
    f[0] = 3.0  (qubit che contribuiscono alla misura finale)
    d[0] = 4.0  (totale qubit in input)

  Sottocircuito 1:
    Vuoto
  Sottocircuito 2:
    Vuoto
  Sottocircuito 3:
    a[3] = 3.0  (gate/qubit originali)
    p